In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math as math
import torch
import test_fn

In [40]:
import importlib
importlib.reload(test_fn)

<module 'test_fn' from 'C:\\Users\\ayeh21\\Documents\\HesBO\\baseline\\test_fn.py'>

# Algorithm 1: Shift or twist subspace to best performing points

In [41]:
importlib.reload(test_fn)
fn = test_fn.Ackley()

low_dim = 2
fn_dim = 200

x = torch.rand((100,low_dim))
A = torch.rand((low_dim, fn_dim))
X = x@A
y = fn.evaluate(X)

# Find best performing points (Alt: Find best performing and nearest neighbors)
best_performers_indices = torch.topk(y, 1, largest = False).indices
best_performers = X[best_performers_indices]

# Update
A = best_performers

In [48]:
def learn_subspace_best_performers(fn, low_dim, fn_dim, 
                                   num_best = 1, num_neighbors = 0,
                                   twist_subspace = True,
                                   twist_step_size = 1,
                                   move_subspace = False,
                                   move_step_size = 1,
                                   restarts = 0,
                                   samples = 200, max_iter = 1000, 
                                   initial_offset = 2, report = True, trace = False):
    
    best_found = 1e10
    A = torch.randn((low_dim, fn_dim))
    B = torch.randn(fn_dim) + initial_offset
    first_run = True
    
    for i in range(max_iter):
        # Evaluate
        x = (torch.rand((samples,low_dim)) * 2) - 1
        X = x@A + B
        y = fn.evaluate(X)

        # Find best performing points (Alt: Find best performing and nearest neighbors)
        best_performers_indices = torch.topk(y, num_best, largest = False).indices
        
        if num_neighbors > 0:
            for j in best_performers_indices:
                distances = torch.sum((X - X[j,:])**2, dim = 1)
                neighbor_indices = torch.topk(distances, num_neighbors, largest = False).indices
                neighbor_indices = neighbor_indices[1:]
                best_performers_indices = torch.concat([best_performers_indices, neighbor_indices])
        
        shift_point = X[best_performers_indices]
        
        

        # Update
        if twist_subspace:
            A = torch.concat([shift_point * twist_step_size, torch.randn((low_dim - num_best, fn_dim))], dim = 0)
        else:
            # Else, reinitialize the transition matrix
            A = torch.randn((low_dim, fn_dim))
            
        if move_subspace:
            if trace:
                print(shift_point.shape)
            B = torch.mean(shift_point * move_step_size, dim = 0)

        # Iterate best found (searching for minimum)
        if min(y) < best_found:
            best_point = X[best_performers_indices[0]]
            best_found = min(y)

        if i%int(max_iter/10) == 0 and report == True:
            print(f"Step {i} current min/best min found y is:{min(y):.2f}, {best_found:.2f}")
            
    return best_found
            
#     print(f"Best point found at {best_point}")

In [49]:
results = []
for i in range(10):
    m = learn_subspace_best_performers(fn, 2, 200, 
                                   initial_offset = 2,
                                   num_best = 1, num_neighbors = 0,
                                   twist_subspace = False,
                                   twist_step_size = 0,
                                   move_subspace = True,
                                   move_step_size = 1,
                                   restarts = 0,
                                   report = False)
    results.append(m)

Mean: {np.mean(results)}
Mean: {np.mean(results)}
Mean: {np.mean(results)}
Mean: {np.mean(results)}
Mean: {np.mean(results)}
Mean: {np.mean(results)}
Mean: {np.mean(results)}
Mean: {np.mean(results)}
Mean: {np.mean(results)}
Mean: {np.mean(results)}


In [50]:
print(f"Mean: {np.mean(results)}")

Mean: 6.281429290771484


In [53]:
results = []
for i in range(10):
    m = learn_subspace_best_performers(fn, 2, 200, 
                                   initial_offset = 50,
                                   num_best = 1, num_neighbors = 0,
                                   twist_subspace = False,
                                   twist_step_size = 0,
                                   move_subspace = True,
                                   move_step_size = 1,
                                   restarts = 0,
                                   report = False)
    results.append(m)

In [54]:
print(f"Mean: {np.mean(results)}")

Mean: 15.757272720336914


In [55]:
results = []
for i in range(10):
    m = learn_subspace_best_performers(fn, 2, 200, 
                                   initial_offset = 50,
                                   num_best = 1, num_neighbors = 0,
                                   twist_subspace = True,
                                   twist_step_size = 0.5,
                                   move_subspace = True,
                                   move_step_size = 0.5,
                                   restarts = 0,
                                   report = False)
    results.append(m)

In [56]:
print(f"Mean: {np.mean(results)}")

Mean: 0.0005502700805664062


In [57]:
results

[tensor(0.0004),
 tensor(0.0006),
 tensor(0.0004),
 tensor(0.0005),
 tensor(0.0006),
 tensor(0.0008),
 tensor(0.0006),
 tensor(0.0003),
 tensor(0.0008),
 tensor(0.0005)]

In [63]:
results = []
for i in range(10):
    m = learn_subspace_best_performers(fn, 2, 200, 
                                   initial_offset = -100,
                                   num_best = 1, num_neighbors = 0,
                                   twist_subspace = True,
                                   twist_step_size = 0.5,
                                   move_subspace = True,
                                   move_step_size = 0.5,
                                   restarts = 0,
                                   report = False)
    results.append(m)

In [64]:
print(f"Mean: {np.mean(results)}")

Mean: 0.0005853652837686241


In [65]:
results

[tensor(0.0004),
 tensor(0.0003),
 tensor(0.0006),
 tensor(0.0007),
 tensor(0.0004),
 tensor(0.0006),
 tensor(0.0009),
 tensor(0.0007),
 tensor(0.0006),
 tensor(0.0007)]

In [31]:
learn_subspace_best_performers(fn, 50, 200, 
                               num_best = 2, num_neighbors = 0,
                               move_subspace = False,
                               move_step_size = 1,
                               restarts = 0)

Step 0 current min/best min found y is:12.53, 12.53
Step 100 current min/best min found y is:12.83, 11.59
Step 200 current min/best min found y is:13.12, 11.59
Step 300 current min/best min found y is:12.37, 11.51
Step 400 current min/best min found y is:12.55, 11.51
Step 500 current min/best min found y is:12.89, 11.51
Step 600 current min/best min found y is:12.73, 11.51
Step 700 current min/best min found y is:13.03, 11.33
Step 800 current min/best min found y is:12.81, 11.33
Step 900 current min/best min found y is:12.38, 11.33


tensor(11.3344)

In [32]:
learn_subspace_best_performers(fn, 50, 200, 
                               num_best = 2, num_neighbors = 0,
                               twist_subspace = True,
                               move_subspace = False,
                               twist_step_size = 0.5,
                               restarts = 0)

Step 0 current min/best min found y is:12.23, 12.23
Step 100 current min/best min found y is:12.77, 11.32
Step 200 current min/best min found y is:12.02, 11.29
Step 300 current min/best min found y is:12.36, 11.29
Step 400 current min/best min found y is:12.19, 11.22
Step 500 current min/best min found y is:12.46, 11.22
Step 600 current min/best min found y is:12.55, 11.22
Step 700 current min/best min found y is:12.20, 11.22
Step 800 current min/best min found y is:12.17, 11.22
Step 900 current min/best min found y is:12.42, 11.22


tensor(11.2159)

In [33]:
learn_subspace_best_performers(fn, 50, 200, 
                               num_best = 2, num_neighbors = 1,
                               twist_subspace = False,
                               move_subspace = True,
                               move_step_size = 0.8,
                               max_iter = 10000,
                               restarts = 0)

Step 0 current min/best min found y is:12.53, 12.53
Step 1000 current min/best min found y is:13.07, 11.69
Step 2000 current min/best min found y is:13.03, 11.69
Step 3000 current min/best min found y is:12.96, 11.69
Step 4000 current min/best min found y is:12.93, 11.69
Step 5000 current min/best min found y is:12.02, 11.69
Step 6000 current min/best min found y is:13.13, 11.69
Step 7000 current min/best min found y is:12.74, 11.69
Step 8000 current min/best min found y is:12.71, 11.62
Step 9000 current min/best min found y is:13.06, 11.62


tensor(11.6197)

In [34]:
learn_subspace_best_performers(fn, 4, 200, 
                               num_best = 2, num_neighbors = 0,
                               twist_subspace = False,
                               move_subspace = True,
                               move_step_size = 0.8,
                               max_iter = 10000,
                               restarts = 0)

Step 0 current min/best min found y is:8.92, 8.92
Step 1000 current min/best min found y is:3.31, 2.64
Step 2000 current min/best min found y is:3.54, 2.62
Step 3000 current min/best min found y is:2.84, 2.21
Step 4000 current min/best min found y is:3.61, 2.21
Step 5000 current min/best min found y is:3.22, 1.95
Step 6000 current min/best min found y is:3.58, 1.95
Step 7000 current min/best min found y is:3.63, 1.95
Step 8000 current min/best min found y is:3.82, 1.95
Step 9000 current min/best min found y is:3.60, 1.95


tensor(1.9476)

In [35]:
learn_subspace_best_performers(fn, 4, 200, 
                               num_best = 2, num_neighbors = 0,
                               twist_subspace = True,
                               move_subspace = False,
                               twist_step_size = 0.8,
                               max_iter = 10000,
                               restarts = 0)

Step 0 current min/best min found y is:8.76, 8.76
Step 1000 current min/best min found y is:7.94, 1.37
Step 2000 current min/best min found y is:4.26, 1.37
Step 3000 current min/best min found y is:7.47, 1.37
Step 4000 current min/best min found y is:4.60, 1.37
Step 5000 current min/best min found y is:7.32, 1.37
Step 6000 current min/best min found y is:8.31, 1.37
Step 7000 current min/best min found y is:3.80, 1.37
Step 8000 current min/best min found y is:4.60, 1.37
Step 9000 current min/best min found y is:5.37, 1.37


tensor(1.3690)

In [36]:
learn_subspace_best_performers(fn, 4, 200, 
                               num_best = 2, num_neighbors = 0,
                               twist_subspace = False,
                               move_subspace = True,
                               move_step_size = 0.8,
                               max_iter = 10,
                               restarts = 0)

Step 0 current min/best min found y is:8.82, 8.82
Step 1 current min/best min found y is:7.55, 7.55
Step 2 current min/best min found y is:6.52, 6.52
Step 3 current min/best min found y is:5.64, 5.64
Step 4 current min/best min found y is:4.98, 4.98
Step 5 current min/best min found y is:4.56, 4.56
Step 6 current min/best min found y is:4.33, 4.33
Step 7 current min/best min found y is:3.80, 3.80
Step 8 current min/best min found y is:4.06, 3.80
Step 9 current min/best min found y is:3.75, 3.75


tensor(3.7460)

In [37]:
learn_subspace_best_performers(fn, 4, 200, 
                               num_best = 2, num_neighbors = 0,
                               twist_subspace = False,
                               move_subspace = True,
                               move_step_size = 0.8,
                               max_iter = 5000,
                               restarts = 0,
                               initial_offset = 50)

Step 0 current min/best min found y is:21.57, 21.57
Step 500 current min/best min found y is:3.48, 2.51
Step 1000 current min/best min found y is:3.41, 2.43
Step 1500 current min/best min found y is:3.79, 2.43
Step 2000 current min/best min found y is:3.56, 2.43
Step 2500 current min/best min found y is:3.57, 2.43
Step 3000 current min/best min found y is:3.20, 2.43
Step 3500 current min/best min found y is:3.12, 2.42
Step 4000 current min/best min found y is:3.34, 2.42
Step 4500 current min/best min found y is:3.41, 2.42


tensor(2.4211)

In [38]:
learn_subspace_best_performers(fn, 4, 200, 
                               num_best = 2, num_neighbors = 0,
                               twist_subspace = True,
                               move_subspace = False,
                               twist_step_size = 0.8,
                               max_iter = 5000,
                               restarts = 0,
                               initial_offset = 50)

Step 0 current min/best min found y is:21.57, 21.57
Step 500 current min/best min found y is:21.57, 3.84
Step 1000 current min/best min found y is:21.57, 2.78
Step 1500 current min/best min found y is:21.57, 2.78
Step 2000 current min/best min found y is:21.57, 2.78
Step 2500 current min/best min found y is:21.57, 2.78
Step 3000 current min/best min found y is:21.57, 2.78
Step 3500 current min/best min found y is:21.57, 2.78
Step 4000 current min/best min found y is:21.57, 2.78
Step 4500 current min/best min found y is:21.57, 2.78


tensor(2.7781)